# Выбор целевой переменной

## Забираем исторические данные

И немного их сглаживаем.

In [ ]:
import datetime

import plotly.graph_objects as go
import pandas as pd

from acquisition import fetch_candle_data
from processing.plot_utils import make_candlestick

first_date = datetime.date(year=2019, month=1, day=1)
last_date = datetime.date(year=2021, month=7, day=1)

candle_data = fetch_candle_data(
    ticker='NVDA',
    first_date=first_date,
    last_date=last_date)

raw_close: 'pd.Series[float]' = candle_data.c
smoothed_close: 'pd.Series[float]' = raw_close.rolling(window=3, win_type='gaussian').mean(std=0.9)

candles_fig = go.FigureWidget(make_candlestick(candle_data[30:200]))
close_fig = go.FigureWidget()
close_fig.add_scatter(x=candle_data.t[30:200], y=raw_close[30:200], name='raw close')
close_fig.add_scatter(x=candle_data.t[30:200], y=smoothed_close[30:200], name='smoothed close')

candles_fig.show()
close_fig.show()


## Целевая переменная №1

### Входные параметры

* $T_b$ - порог покупки $[\%]$
* $T_s$ - порог продажи $[\%]$
* $T_{nothing}$ - порог "ничего не делать" $[\%]$
* $W$ - размер окна $[-]$


### Алгоритм

1. Для каждого значения цены из окна посчитать относительную прибыль:

$$Return_i = \frac{Close_{N+i} - Close_N}{Close_N}$$
$$i \in \left[1;W\right)$$

2. Посчитать внутри окна вероятность появления возможности продать позицию дороже на $T_b$, либо купить позицию на $T_s$ дешевле:

$$P_b = P\{Return_i > T_b\}$$
$$P_s = P\{Return_i < -T_s\}$$

3.
    * Если $\left|P_b - P_s\right| < T_{nothing}$, то нет ни сигнала на покупку, ни сигнала на продажу
    * Иначе, если $P_b > P_s$, то есть сигнал на покупку
    * Иначе, $P_s > P_b$, есть сигнал на продажу


In [ ]:
import typing

import pandas as pd


def target_strategy_1(
        close: 'pd.Series[float]',
        buy_threshold: float,
        sell_threshold: float,
        do_nothing_threshold: float) -> float:
    current_price = typing.cast(float, close.values[0])

    buy_freq = 0
    sell_freq = 0

    for idx in range(1, len(close)):
        ret = (
            typing.cast(
                float,
                close.values[idx]) - current_price) / current_price
        if ret > buy_threshold:
            buy_freq += 1
        if ret < -sell_threshold:
            sell_freq += 1

    signal_strength = abs(buy_freq - sell_freq) / (len(close) - 1)
    if signal_strength < do_nothing_threshold:
        return 0.0

    if buy_freq > sell_freq:
        return buy_freq / (len(close) - 1)
    if sell_freq > buy_freq:
        return -sell_freq / (len(close) - 1)

    return 0.0


### Ищем хорошие параметры для целевой переменной

TODO:
* использовать оптимизацию Байеса по всем 4 параметрам 

In [ ]:
import functools

import numpy as np
import plotly.graph_objects as go

from processing.backtest import backtest_target_strategy
from processing.target import eval_target

buy_threshold = np.linspace(0.0, 0.045, 15)
sell_threshold = np.linspace(0.0, 0.045, 15)

values = np.ndarray((len(sell_threshold), len(buy_threshold)))

time_delta = (candle_data.t.values[-1] - candle_data.t.values[0]
              ).astype('timedelta64[D]').astype(int)

for buy_idx in range(0, len(buy_threshold)):
    b = buy_threshold[buy_idx]

    for sell_idx in range(0, len(sell_threshold)):
        s = sell_threshold[sell_idx]
        strategy = functools.partial(
            target_strategy_1,
            buy_threshold=b,
            sell_threshold=s,
            do_nothing_threshold=0.3)

        target_pre_smooth = eval_target(smoothed_close, 7, strategy)

        cash_flow, return_history = backtest_target_strategy(
            close=candle_data.c, target=target_pre_smooth.actions)

        rating = (
            return_history.values[-1] - return_history.values[0]) / return_history.values[0]

        values[sell_idx][buy_idx] = rating * 365 / time_delta * 100

heatmap_fig = go.FigureWidget()
heatmap_fig.add_heatmap(z=values, x=buy_threshold, y=sell_threshold)
heatmap_fig.update_layout(
    title='Target Return [%]',
    xaxis_title='Buy Threshold',
    yaxis_title='Sell Threshold'
)
heatmap_fig.show()


### Выбрали параметры:

$W = 7, \space\space T_s = 0.32\%, \space\space T_b = 0.64\%, \space\space T_{nothing} = 30\%$

In [ ]:
window = 7
buy_threshold = 0.0064
sell_threshold = 0.0032
do_nothing_threshold = 0.3

strategy = functools.partial(
    target_strategy_1,
    buy_threshold=buy_threshold,
    sell_threshold=sell_threshold,
    do_nothing_threshold=do_nothing_threshold)

target = eval_target(smoothed_close, window, strategy)

cash_flow, return_history = backtest_target_strategy(
    close=candle_data.c, target=target.actions)

return_fig = go.FigureWidget()
return_fig.add_scatter(x=candle_data.t, y=return_history, name='target')

return_rate = (return_history.values[-1] -
               return_history.values[0]) / return_history.values[0]

print(f'Return rate: {return_rate / time_delta * 365 * 100}%/year')

buy_and_hold_target = pd.Series(index=smoothed_close.index, dtype=float)
buy_and_hold_target.fillna(0.0)
buy_and_hold_target[0] = 1

cash_flow, return_history = backtest_target_strategy(
    close=candle_data.c, target=buy_and_hold_target)

return_rate = (
    return_history.values[-1] - return_history.values[0]) / return_history.values[0]

print(f'Buy and hold return rate: {return_rate/ time_delta * 365 * 100}%/year')

return_fig.add_scatter(x=candle_data.t, y=return_history, name='buy and hold')
return_fig.show()
